# The Workflow - Un-Agentified

- Let's checkout what the raw workflow looks like. 

 ## Importing all necessary modules.

In [1]:
import os, json, logging
from ollama import chat
from ollama import ChatResponse
#Manage Environment Variables. 
from dotenv import load_dotenv
load_dotenv()

auradb_conn=os.environ["AURADB_CONN"]
auradb_username = os.environ["AURADB_USERNAME"]
auradb_password = os.environ["AURADB_PASSWORD"]

# Generate chat response using local LLMs

- We've installed Ollama locally to get access to cutting edge instruct models. 
- The ollama interface allows us to generate responses by loading these large language models locally.
- In this case we'll check out what kind of prompts we'll be giving models as inputs to generate outputs we can utilize. 

In [3]:
passage = """
In the tropical rainforest ecosystem, several species exhibit a variety of biological relationships that illustrate evolutionary, ecological, and taxonomic connections. The Jaguar (Panthera onca) is a top predator and preys upon the Capybara (Hydrochoerus hydrochaeris) and the Green Anaconda (Eunectes murinus). Both the Capybara and Green Anaconda share the same wetland habitat, demonstrating ecological co-occurrence.
The Green Anaconda often competes with the Harpy Eagle (Harpia harpyja) for prey such as the Howler Monkey (Alouatta palliata). The Howler Monkey, an arboreal primate, is closely related to the Spider Monkey (Ateles geoffroyi), with both belonging to the family Atelidae. This taxonomic relationship indicates a common evolutionary ancestor.
The Harpy Eagle also preys on the Sloth (Bradypus variegatus), which has a mutualistic relationship with several species of algae, such as Trentepohlia spp., that grow on its fur, providing camouflage. Additionally, the Sloth shares a symbiotic relationship with the Moth (Cryptoses choloepi), which uses the sloth’s fur for habitat.
From an evolutionary standpoint, the Capybara and the Jaguar belong to different orders: Rodentia and Carnivora, respectively, reflecting deep phylogenetic divergence. The Sloth is taxonomically grouped within the order Pilosa, distinct from the other mammals mentioned.
Together, these species form a complex web of predation, competition, symbiosis, and shared evolutionary history, making the tropical rainforest a dynamic and interconnected ecosystem.
"""

In [8]:
prompt = f"""For the following passage, give me ONLY a serialized json object that 
describes the relationship between the species, compatible with a graph data structure. 
For example a graph relationship: Lion --> (predator) --> Antelope should output a serialized json string \n

the passage starts from the next line \n {passage} \n


*** Requirements ***

- Only return the json object. Nothing else.
- each node relationship should have only one source or target. Not a list. 

*** Example output for a relationship ***

{{
       "node": "Jaguar",
       "relationships": [
         {{
           "type": "predator",
           "target": "Capybara"
         }}
       ]
     }}

"""

response: ChatResponse = chat(model='mistral:instruct', messages=[
  {
    'role': 'user',
    'content': prompt
  },
    
], stream=True)

result=""
for chunk in response:
    print(chunk['message']['content'], end='', flush=True)  # Keep output on same line
    result+= str(chunk['message']['content'])
    
print()



 [
      {
        "node": "Jaguar",
        "relationships": [
          {
            "type": "predator",
            "target": "Capybara"
          },
          {
            "type": "predator",
            "target": "Green Anaconda"
          }
        ]
      },
      {
        "node": "Capybara",
        "relationships": []
      },
      {
        "node": "Green Anaconda",
        "relationships": [
          {
            "type": "competitor",
            "target": "Harpy Eagle"
          }
        ]
      },
      {
        "node": "Harpy Eagle",
        "relationships": [
          {
            "type": "predator",
            "target": "Green Anaconda"
          },
          {
            "type": "predator",
            "target": "Howler Monkey"
          }
        ]
      },
      {
        "node": "Howler Monkey",
        "relationships": [
          {
            "type": "related",
            "target": "Spider Monkey"
          },
          {
            "type": "predato

In [9]:
obj = json.loads(result)
print(obj)
print(type(obj))

[{'node': 'Jaguar', 'relationships': [{'type': 'predator', 'target': 'Capybara'}, {'type': 'predator', 'target': 'Green Anaconda'}]}, {'node': 'Capybara', 'relationships': []}, {'node': 'Green Anaconda', 'relationships': [{'type': 'competitor', 'target': 'Harpy Eagle'}]}, {'node': 'Harpy Eagle', 'relationships': [{'type': 'predator', 'target': 'Green Anaconda'}, {'type': 'predator', 'target': 'Howler Monkey'}]}, {'node': 'Howler Monkey', 'relationships': [{'type': 'related', 'target': 'Spider Monkey'}, {'type': 'predator', 'target': 'Harpy Eagle'}]}, {'node': 'Spider Monkey', 'relationships': [{'type': 'related', 'target': 'Howler Monkey'}]}, {'node': 'Sloth', 'relationships': [{'type': 'symbiont', 'source': 'Trentepohlia spp.'}, {'type': 'habitat_provider', 'target': 'Moth'}, {'type': 'predator', 'target': 'Harpy Eagle'}]}, {'node': 'Trentepohlia spp.', 'relationships': [{'type': 'symbiont', 'target': 'Sloth'}]}, {'node': 'Moth', 'relationships': [{'type': 'habitat_provider', 'source'

# Initializing a Graph Database using AuraDB

In [10]:
from neo4j import GraphDatabase, ResultSummary
AURA_CONNECTION_URI = auradb_conn
AURA_USERNAME = "neo4j"
AURA_PASSWORD = auradb_password

# Driver instantiation
driver = GraphDatabase.driver(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD)
)

import uuid

input_data = obj  # Your provided JSON here

# Transform nodes
nodes = {item["node"]: {"id": str(uuid.uuid4())} for item in input_data}

print(nodes)
# Transform relationships
relationships = []
for item in input_data:
    source_node = item["node"]
    for rel in item["relationships"]:
        print(rel)
        relationships.append({
            "source": source_node,
            "target": rel.get("target", rel.get("source")),
            "type": rel["type"].upper().replace(" ", "_")
        })
print(f"Relationships created: {relationships}")

def create_graph(tx):
    # Create nodes with unique IDs
    for name, props in nodes.items():
        nodes_result = tx.run("""
            MERGE (n:Species {name: $name})
            SET n.id = $id
        """, name=name, id=props["id"])

    # Create relationships
    for rel in relationships:
        relationships_result = tx.run("""
            MATCH (a:Species {name: $source}), (b:Species {name: $target})
            MERGE (a)-[r:RELATES_TO {type: $type}]->(b)
        """, source=rel["source"], target=rel["target"], type=rel["type"])
    
    return {"nodes": nodes_result.consume(), "relationships": relationships_result.consume()}

try:
    with driver.session() as session:
        summary = session.execute_write(create_graph)
        
            
        print({
                "success": True,
                "message": "Graph push successful",
                "nodes": summary["nodes"].counters,
                "relationships": summary["relationships"].counters
            })
        
except Exception as e:
    print({
            "success": False,
            "message": "Graph push failed",
            "summary": str(e)
            })
    

{'Jaguar': {'id': 'd3491f89-0bd6-4e1b-8d15-73753e6ade3a'}, 'Capybara': {'id': '9034293d-aa7f-49ae-9656-c51805199daf'}, 'Green Anaconda': {'id': '20179fcd-a2bd-49b9-8c6b-c4a97eaee7cf'}, 'Harpy Eagle': {'id': '6ab3b68b-0395-49fb-8fd8-8509867e0fc7'}, 'Howler Monkey': {'id': 'c0fb22f2-bf51-43b5-ac5a-a211e46078e7'}, 'Spider Monkey': {'id': '44aff8b1-1ce7-437d-b0f9-a2e1cd20afb4'}, 'Sloth': {'id': 'd7daef90-e9ed-4079-8b95-22a66e6cb55b'}, 'Trentepohlia spp.': {'id': 'c796cdc6-2671-4cb2-bcdf-fd59e5488371'}, 'Moth': {'id': '7e9a04a7-5923-4776-ac28-6e5b84545fea'}}
{'type': 'predator', 'target': 'Capybara'}
{'type': 'predator', 'target': 'Green Anaconda'}
{'type': 'competitor', 'target': 'Harpy Eagle'}
{'type': 'predator', 'target': 'Green Anaconda'}
{'type': 'predator', 'target': 'Howler Monkey'}
{'type': 'related', 'target': 'Spider Monkey'}
{'type': 'predator', 'target': 'Harpy Eagle'}
{'type': 'related', 'target': 'Howler Monkey'}
{'type': 'symbiont', 'source': 'Trentepohlia spp.'}
{'type': 'h